# Discontinuous Galerkin Methods

* Use discontinuous finite element spaces to solve PDEs. 
* Allows upwind-stabilization for convection-dominated problems
* Requires additional jump terms for consistency 

Interior penalty DG form for $-\Delta u$:

$$
\DeclareMathOperator{\Div}{div}
A(u,v) = \sum_T \int_T \nabla u \nabla v
-  \sum_F \int_F \{ n \nabla u \} [v] 
-  \sum_F \int_F \{ n \nabla v \} [u] 
+ \frac{\alpha p^2}{h} \sum_F \int_F [u][v]
$$

with jump-term over facets:
$$
[u] = u_{left} - u_{right}
$$

and averaging operator
$$
\{ n \nabla u \} = \tfrac{1}{2} (n_{left} \nabla u_{left} + n_{left} \nabla u_{right})
$$

DG form for $\Div (b u)$, where $b$ is the given wind:

$$
B(u,v) = -\sum_T b u \nabla v + \sum_F \int_F b\cdot n   u^{upwind} v 
$$

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

## A periodic mesh

In [ ]:
shape = Rectangle(1,1).Face()

shape.edges.Max(X).name = "right"
shape.edges.Min(X).name = "left"
shape.edges.Max(Y).name = "top"
shape.edges.Min(Y).name = "bot"

shape.edges.Max(Y).Identify(shape.edges.Min(Y), "bt")
shape.edges.Max(X).Identify(shape.edges.Min(X), "lr")

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.1))

In [ ]:
plist = []
for pair in mesh.ngmesh.GetIdentifications():
    plist += list(mesh.vertices[pair[0]-1].point) + [0]
    plist += list(mesh.vertices[pair[1]-1].point) + [0]
Draw(mesh, objects=[{"type" : "lines", "position" : plist, "name": "identification", "color": "purple"}]);

## The broken polynomial space
The space is responsible for allocating the matrix graph. Tell it that it should reserve entries for the coupling terms:

In [ ]:
order=4
fes = Periodic(L2(mesh, order=order, dgjumps=True))
u,v = fes.TnT()

Every facet has a master element. The value from the other element is referred to via the
`Other()` operator:

In [ ]:
jump_u = u-u.Other()
jump_v = v-v.Other()
n = specialcf.normal(2)
mean_dudn = 0.5*n * (grad(u)+grad(u.Other()))
mean_dvdn = 0.5*n * (grad(v)+grad(v.Other()))

Integrals on facets are computed by setting `skeleton=True`. 
* `dx(skeleton=True)` iterates over all internal faces

In [ ]:
alpha = 4
h = specialcf.mesh_size
diffreact = grad(u)*grad(v) * dx + u*v*dx \
    +alpha*order**2/h*jump_u*jump_v * dx(skeleton=True) \
    +(-mean_dudn*jump_v-mean_dvdn*jump_u) * dx(skeleton=True) 

a = BilinearForm(diffreact).Assemble()

In [ ]:
f = LinearForm(exp(x**2+y**2)*v*dx).Assemble()

In [ ]:
gfu = GridFunction(fes, name="uDG")
gfu.vec.data = a.mat.Inverse() * f.vec
Draw (gfu);

DG requires a lot of additional matrix entries:

In [ ]:
fes2 = L2(mesh, order=order)
ul2,vl2 = fes2.TnT()
a2 = BilinearForm(ul2*vl2*dx).Assemble()
print ("DG-matrix nze:", a.mat.nze)
print ("L2-matrix nze:", a2.mat.nze)

#### Try adding some convection and upwinding term:
The `IfPos` checks whether the first argument is positive. Then it returns the second one, else the third one. This is used to define the upwind flux. The check is performed in every integration-point on the skeleton:

In [ ]:
b = CF( (20,5) )
#.....

### Boundary conditions are a piece of cake!
* `ds(skeleton=True)` iterates over all boundary faces

In [ ]:
fa = WorkPlane(Axes((0,0,0), Y,X)).MoveTo(0.3,0).Rectangle(3,1).Face()
ax = Axis ((0,0,0), Z)
cake = fa.Revolve(ax, 30)
cake.faces.Min(Y).name="f1"
cake.faces.Max(Y-0.5*X).name="f2"
cake.faces.Min(Z).name="bot"

cake.faces["f1"][0].Identify(cake.faces["f2"][0], "id",
                            trafo=Rotation(ax, 30))
Draw (cake);
## periodic
mesh = Mesh(OCCGeometry(cake).GenerateMesh(maxh=0.5)) # .Curve(3)
mesh.ngmesh.Refine()
plist = []
for pair in mesh.ngmesh.GetIdentifications():
    plist += list(mesh.vertices[pair[0]-1].point)
    plist += list(mesh.vertices[pair[1]-1].point)
#Draw(mesh, objects=[{"type" : "lines", "position" : plist, "name": "identification", "color": "purple"}]);

#### Try implement Dirichlet boundary conditions on "bot"
* use `ds(skeleton=True, definedon=mesh.Boundaries("bndname"))`for part of the boundary

### Remarks on sparsity pattern in NGSolve

### Remark: The sparsity pattern is set up a-priorily
* The sparsity pattern of a sparse matrix in NGSolve is independent of its entries (it's set up a-priorily). 
* We can have "nonzero" entries that have the value 0

Below we show the reserved memory for the sparse matrix and the (numerically) non-zero entries in this sparse matrix. 

In [ ]:
fes2 = L2(mesh, order=order, dgjumps=True)
u,v=fes2.TnT()
a3 = BilinearForm(fes2)
a3 += u*v*dx + (u+u.Other())*v*dx(skeleton=True)
a3.Assemble();

In [ ]:
import scipy.sparse as sp
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (12, 12)
A = sp.csr_matrix(a3.mat.CSR())
fig = plt.figure(); ax1 = fig.add_subplot(121); ax2 = fig.add_subplot(122)
ax1.set_xlabel("numerically non-zero"); ax1.spy(A)
ax2.set_xlabel("reserved entries (potentially non-zero)"); ax2.spy(A,precision=-1)
plt.show()

#### Remark 2: Sparsity pattern with and without `dgjumps=True` is different

In [ ]:
a1 = BilinearForm(L2(mesh, order=order, dgjumps=False)); a1.Assemble()
a2 = BilinearForm(L2(mesh, order=order, dgjumps=True)); a2.Assemble()
A1 = sp.csr_matrix(a1.mat.CSR())
A2 = sp.csr_matrix(a2.mat.CSR())
fig = plt.figure(); ax1 = fig.add_subplot(121); ax2 = fig.add_subplot(122)
ax1.set_xlabel("dgjumps=False"); ax1.spy(A1,precision=-1)
ax2.set_xlabel("dgjumps=True"); ax2.spy(A2,precision=-1)
plt.show()